# "Intuitively understanding Python decorators"
> "Syntactic sugar and functional implementation of decorators"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [Python, functional programming]
- image: images/decorators/decorator.jpg
- hide: false
- search_exclude: true

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

## Functions
Before you can understand decorators, you must first understand how functions work. For our purposes, a function returns a value based on the given arguments. Here is a very simple example:

In [3]:
def add_one(number): return number+1

add_one(2)

3

In general, functions in Python may also have side effects rather than just turning an input into an output. The `print()` function is a basic example of this: it returns `None` while having the side effect of outputting something to the console. However, to understand decorators, it is enough to think about functions as something that turns given arguments into a value.

### First-class objects
In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (`string`, `int`, `float`, `list`, and so on). Consider the following three functions:

In [7]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, `say_hello()` and `be_awesome()` are regular functions that expect a name given as a string. The `greet_bob()` function however, expects a function as its argument. We can, for instance, pass it the `say_hello()` or the `be_awesome()` function:

In [8]:
greet_bob(say_hello)

'Hello Bob'

In [9]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

Note that `greet_bob(say_hello)` refers to two functions, but in different ways: `greet_bob()` and `say_hello`. The `say_hello` function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The `greet_bob()` function, on the other hand, is written with parentheses, so it will be called as usual.

### Inner-functions
It’s possible to define functions inside other functions. Such functions are called _inner functions_. Here’s an example of a function with two inner functions:

In [12]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

What happens when you call the `parent()` function? Think about this for a minute. The output will be as follows:

In [13]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Note that the order in which the inner functions are defined does not matter. Like with any other functions, the printing only happens when the inner functions are executed.

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to `parent()`: they only exist inside the `parent()` function as local variables. Try calling `first_child()`. You should get an error:

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to `parent()`: they only exist inside the `parent()` function as local variables. Try calling `first_child()`. You should get an error:

In [15]:
# collapse
first_child()

NameError: name 'first_child' is not defined

Whenever you call `parent()`, the inner functions `first_child()` and `second_child()` are also called. But because of their local scope, they aren’t available outside of the `parent()` function.

### Returning functions from functions
Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer `parent()` function:

In [16]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

Note that you are returning `first_child` without the parentheses. Recall that this means that you are returning a reference to the function `first_child`. In contrast `first_child()` with parentheses refers to the result of evaluating the function. This can be seen in the following example:

In [17]:
first = parent(1)
second = parent(2)

In [18]:
first

<function __main__.parent.<locals>.first_child()>

In [20]:
second

<function __main__.parent.<locals>.second_child()>

The somewhat cryptic output simply means that the first variable refers to the local `first_child()` function inside of `parent()`, while second points to `second_child()`.

You can now use first and second as if they are regular functions, even though the functions they point to can’t be accessed directly:

In [21]:
first()

'Hi, I am Emma'

In [22]:
second()

'Call me Liam'

Finally, note that in the earlier example you executed the inner functions within the parent function, for instance `first_child()`. However, in this last example, you did not add parentheses to the inner functions — `first_child` — upon returning. That way, you got a reference to each function that you could call in the future. Make sense?

## Simple decorators
Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [23]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [24]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


To understand what’s going on here, look back at the previous examples. We are literally just applying everything you have learned so far.

The so-called decoration happens at the following line:
```{python}
say_whee = my_decorator(say_whee)
```

In effect, the name `say_whee` now points to the `wrapper()` inner function. Remember that you return wrapper as a function when you call `my_decorator(say_whee)`:

In [25]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

However, `wrapper()` has a reference to the original `say_whee()` as func, and calls that function between the two calls to `print()`.

Put simply: __decorators wrap a function, modifying its behavior__.

Before moving on, let’s have a look at a second example. Because `wrapper()` is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [26]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

If you try to call `say_whee()` after bedtime, nothing will happen:

## Syntactic sugar
